In [26]:
import pandas as pd

def GenerateSKLearnModelList():
    
    df_estimators = pd.DataFrame(all_estimators(type_filter=None), columns=['Model Name', 'Estimator Class'])
    split_columns = df_estimators['Estimator Class'].astype(str).str.split('.', expand=True)

    # Rename columns dynamically based on the number of splits
    split_columns.columns = [f'Part_{i+1}' for i in range(split_columns.shape[1])]
    # Concatenate the original DataFrame with the split columns
    df_estimators = pd.concat([df_estimators, split_columns], axis=1).drop('Part_1',axis=1)
    df_estimators.to_csv('SKLearnModels.csv',index=False)

In [34]:
import pandas as pd
import numpy as np

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

from MLPipeline import apply_scaling


]

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,Target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [45]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import time

from sklearn import datasets
diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes['data'],columns=diabetes['feature_names'])
df = pd.concat([df,pd.DataFrame(diabetes['target'],columns=['Target'])],axis=1)
df.head()

def MLPipeline(df, 
               project_name,
               scaler,
               ml_model_type='regressor',
               target_column='Target',
               test_size=0.2):
    """
    Runs multiple ML algorithms, tracks results with MLflow, and saves models.

    Args:
        df (dataframe)
        project_name (str):
        scaler (str): None, normal,standard
        ml_model_type (str): Option to pronpt all_estimators as to what model type requested.
        classifier, regressor, cluster, transformer

    Returns:
        None (Results are logged in MLflow)
    """
    # Set up MLflow experiment
    mlflow.set_experiment(project_name)

    # Prepare data
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    # Apply Scaler as Necessary
    X_train, X_test = apply_scaling(X_train, X_test, scaler=scaler)
    
    # Get all available models
    model_list = all_estimators(type_filter=ml_model_type)
    
    results = []
    
    for name, model_class in model_list:
        try:
            model = model_class()
            start_time = time.time()

            with mlflow.start_run(run_name=name):  # Start MLflow run
                model.fit(X_train, y_train)  # Train model
                y_pred = model.predict(X_test)  # Predict

                # Evaluate performance
                if ml_model_type == "classifier":
                    metric = accuracy_score(y_test, y_pred)
                    mlflow.log_metric("Accuracy", metric)
                else:
                    metric = mean_squared_error(y_test, y_pred) ** 0.5  # RMSE manually computed
                    mlflow.log_metric("RMSE", metric)

                # Log model
                mlflow.sklearn.log_model(model, name)

                # Log metadata
                mlflow.log_param("Model", name)
                mlflow.log_param("Training Time", round(time.time() - start_time, 2))

                # Append results
                results.append({
                    "Model": name,
                    "Metric": metric,
                    "Time (s)": round(time.time() - start_time, 2)
                })

        except Exception as e:
            print(f"⚠️ {name} failed: {str(e)}")  # Handle errors but continue

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df#.sort_values(by="Metric", ascending=(task_type == "regression"))

    return results_df
    
MLPipeline(df,'Test',None)

2025/02/24 22:52:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:52:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:52:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:52:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


⚠️ CCA failed: `n_components` upper bound is 1. Got 2 instead. Reduce `n_components`.


2025/02/24 22:52:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:52:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:52:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:52:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:01 

⚠️ IsotonicRegression failed: Isotonic regression input X should be a 1d array or 2d array with 1 feature


2025/02/24 22:53:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:12 

...

2025/02/24 22:53:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/Users/derekdewald/anaconda3/envs/BaseRequirements/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:1608: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
2025/02/24 22:53:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter w

⚠️ StackingRegressor failed: StackingRegressor.__init__() missing 1 required positional argument: 'estimators'


2025/02/24 22:53:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/24 22:53:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


⚠️ VotingRegressor failed: VotingRegressor.__init__() missing 1 required positional argument: 'estimators'


,Model,Metric,Time (s)
0,ARDRegression,53.138882,1.32
1,AdaBoostRegressor,54.118285,1.00
2,BaggingRegressor,56.147850,0.97
3,BayesianRidge,53.588824,0.90
4,DecisionTreeRegressor,69.507739,0.93
5,DummyRegressor,73.222493,0.82
6,ElasticNet,72.878068,0.89
7,ElasticNetCV,54.838504,0.89
8,ExtraTreeRegressor,73.855538,0.92
9,ExtraTreesRegressor,53.615142,1.00


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import all_estimators


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


import matplotlib.pyplot as plt


In [ ]:

# Example usage:


In [ ]:

import mlflow.sklearn
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

from sklearn.utils import all_estimators
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

# Function to run ML pipeline with MLflow tracking


from 



    # Additional models
    extra_models = [
        ("XGBoost", xgb.XGBClassifier() if task_type == "classification" else xgb.XGBRegressor()),
        ("LightGBM", lgb.LGBMClassifier() if task_type == "classification" else lgb.LGBMRegressor()),
        ("CatBoost", cat.CatBoostClassifier(silent=True) if task_type == "classification" else cat.CatBoostRegressor(silent=True))
    ]

    results = []

   
    # Display results in a pandas table
    import ace_tools as tools
    tools.display_dataframe_to_user(name="ML Model Performance with MLflow", dataframe=results_df)

# Example Usage
df = pd.read_csv("path_to_your_dataset.csv")  # Load your dataset
run_ml_pipeline_with_mlflow(df, target_column="target_column_name", task_type="classification")


In [ ]:
def MLManualPipeline(df,
                     X_Cols,
                     y_Col,
                     scaler='MinMaxScaler',
                     model_list=['Linear Regression'],
                     test_size=.3,
                     random_state=42):

    if len(X_Cols) == 0:
        X = np.array(df.drop(y_Col,axis=1).copy())
    else:
        X = np.array(df[X_Cols])
    
    y = df[y_Col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    if scaler =='MinMaxScaler':
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)

    if len(model_list)==0:
        return X_train,X_test,y_train,y_test
    
    else:
        for model in model_list:
            if model == 'Linear Regression':
                lr = LinearRegression()
                lr.fit(X_train, y_train)
                y_pred_lr = lr.predict(X_test)
                
            elif model =='Logistic Regression':
                logreg=LogisticRegression()
                logreg.fit(X_train, y_train)
                y_pred = logreg.predict(X_test)
                print(f"Logisitic Regression Model:\n{confusion_matrix(y_test, y_pred)}\n{classification_report(y_test, y_pred)})")

            elif model =='Random Forest':
                
                ############################################ ESTIMATORS
                rf = RandomForestClassifier(random_state=random_state, n_estimators=25)
                rf.fit(X_train, y_train)
                y_pred_rf = rf.predict(X_test)
                print(f"Random Forest with 25 Nodes?>?>?>:\n{confusion_matrix(y_test, y_pred_rf)}\n{classification_report(y_test, y_pred_rf)})")

MLManualPipeline(df=df3.drop(['MEMBERNBR','ATTRITION_FLAG_1M'],axis=1),
                 X_Cols="",
                 y_Col='ATTRITION_FLAG_2M',
                model_list=['Logistic Regression','Random Forest'])


### Grid Search

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
}

# Grid Search
grid_search = GridSearchCV(
    estimator=xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False),
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# Best parameters and model
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

best_model = grid_search.best_estimator_



## LSTM

In [ ]:
# Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Predict and display results
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Predictions:", y_pred_classes.flatten())
print("Actual:", y_test)

# Plot accuracy and loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.tight_layout()
plt.show()

## Neural Network

In [ ]:
model = keras.models.Sequential()

model.add(layers.Dense(8,activation='relu'))
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(1024, activation='tanh'))
model.add(layers.Dense(2028, activation='tanh'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(32, activation='tanh'))
model.add(layers.Dense(1,activation='softmax'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

# Train the model
model.fit(X_train.fillna(0), y_train, epochs=50, batch_size=1000, validation_split=0.1, verbose=1)



In [ ]:
Feature crosses - add non linear relationships to linear model
Build up intuition gradually by slowly building model up.
Analyze Errors, does it make the mistakes that it’s making.
Accuracy and Error are dependent on a specific chosen threshold and are not differentiable. 
Convexity Good - No matter where you are, you will come to global minimum with following the right direction. 
Multinominal probability distribution - set of numbers equal to 1. 
Most Bias Model - Mean. Simple, Generalizes Well. Undercuts.
Models have noise, if you overfit the noise, it overfits. Variance.
Can perfectly fit n points with Polynomial of N-1. Pure Noise.

- What is your baseline

Output of a prediction, vs output of a bounding window, vs output which highlights area of interest.

Training Population and Usage Population
- NA Testing Group vs Asian Testing Group. Men vs Women. Etc..

Features - Need to generalize. Name of a car vs Engine, Age, Etc.
Turn Learning into a numerical optimization task. Computers really good at this. Opposed to Expert Logic.


Loss Function. 
Model is a function of the inputs. 
Loss is a function of the parameters of the model.
For MSE why a Parabolic Shape? - Squared Function.
Unlikely to generate a single Linear Function that perfectly fits all data points. Need to generate some super complex function
Positive Gradient - Reduce
Negative Gradient - Increase
Convergence / divergence
Greater the slope, the more we want to change

Global vs Local Minimum. No great solution.
Linear regression is convex, always get to global minima

When predicting a car, you need to find common features which generalize well.

Linear Model limitations
- Sensitive to size, color, orientation
- No concept of edges
- No concept of structure relationship between pixels

Edge detection
- Sharp change pixel intensity
- Spatial gradient
- Convolution